# Sample Analysis

4 different seeds with the following:

* Seed A: 0
* Seed B: 42
* Seed C: 123
* Seed D: 2025

652 samples with the following:

* TP: 163
* TN: 163
* FP: 163
* FN: 163

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dtale
import matplotlib.pyplot as plt
import seaborn as sns
import re


from master_thesis.config import PROCESSED_DATA_DIR, load_dataframe_from_pickle, save_dataframe_as_pickle

In [3]:
working_df = load_dataframe_from_pickle(
    PROCESSED_DATA_DIR / "samples_with_XAI_applied.pkl"
)

working_df.head(5)

,text,error,cardiff_label,cardiff_score,lime_cardiff,shap_cardiff,seed
0,"India is one nasty, dirty, filthy, germsy, law...",NaN,HATE,0.994301,"country: -0.137, India: -0.111, filthy: -0.098...","India : -0.138, is : -0.029, one : 0.085, nast...",0
1,solo que seras bombardeas y destruidos como lo...,NaN,HATE,0.957913,"virus: -0.644, a: 0.284, kike: -0.249, solo: -...","s: -0.033, olo : -0.014, que : -0.002, ser: -0...",0
2,@user @user You cannot argue with a retard jer...,NaN,HATE,0.994864,"retard: -0.644, beggar: -0.128, jerusalemofiro...","@: 0.015, user : 0.069, @: -0.003, user : 0.03...",0
3,Can I just pound that pussy an pull your hair....,NaN,HATE,0.977588,"your: -0.585, tits: -0.165, hair: -0.141, on: ...","Can : 0.012, I : -0.012, just : -0.021, pound ...",0
4,you do not live in america nazi faggot but you...,NaN,HATE,0.985650,"faggot: -0.896, nazi: -0.056, young: 0.034, do...","you : 0.001, do : 0.003, not : -0.003, live : ...",0


In [6]:
working_df['error'].value_counts()

Series([], Name: count, dtype: int64)